# Recomendação de música

Carregando dados


In [2]:
import pandas as pd
from scipy.spatial.distance import cosine

df_musicas = pd.read_csv("musicas.csv")
df_dados = pd.read_csv("dados.csv")

Mrs = pd.read_csv("musicas_rs.csv")
Drs = pd.read_csv("dados_rs.csv")

Mminmax = pd.read_csv("musicas_minmax.csv")
Dminmax = pd.read_csv("dados_minmax.csv")

Mmaxabs = pd.read_csv("musicas_maxabs.csv")
Dmaxabs = pd.read_csv("dados_maxabs.csv")

Mstand = pd.read_csv("musicas_standart.csv")
Dstand = pd.read_csv("dados_standart.csv")



Lista de todos os artistas presentes no dataset

In [3]:
todos_artistas = df_musicas['artists'].str.split(';').explode().str.strip()
artistas_unicos = sorted(todos_artistas.unique())
pd.Series(artistas_unicos).to_csv("lista_artistas.csv", index=False)

Funções para recomendação de música

In [4]:
from scipy.spatial.distance import euclidean, cosine

def recomendar_musicas(track_id, df_dados_normalizados, df_musicas):
    idx = df_musicas.index[df_musicas['track_id'] == track_id][0]
    cluster_id = df_musicas.loc[idx, 'cluster']
    imprimir = df_musicas[df_musicas['track_id']== track_id][['artists','track_name']]
    print(f"Música: {imprimir.iloc[0]['track_name']} - {imprimir.iloc[0]['artists']}")
    musica_base = df_dados_normalizados.drop(columns='cluster').iloc[idx].values

    subset = df_dados_normalizados[df_dados_normalizados['cluster'] == cluster_id].copy()
    subset['distancia'] = subset.drop(columns='cluster').apply(lambda row: cosine(row.values, musica_base), axis=1)

    resultado_idx = subset.sort_values('distancia').index
    recomendacoes = df_musicas.loc[resultado_idx][['track_name', 'artists','track_genre']]
    recomendacoes = recomendacoes[recomendacoes.index != idx]

    return recomendacoes.head(10)


In [5]:
def recomendar_musicas_por_genero(track_id, df_dados_normalizados, df_musicas):
    # Alinhar índices
    df_dados_normalizados = df_dados_normalizados.reset_index(drop=True)
    df_musicas = df_musicas.reset_index(drop=True)

    idx = df_musicas.index[df_musicas['track_id'] == track_id][0]
    cluster_id = df_musicas.loc[idx, 'cluster']
    genero_base = df_musicas.loc[idx, 'track_genre']
    imprimir = df_musicas[df_musicas['track_id']== track_id][['artists','track_name','track_genre']]
    print(f"Música: {imprimir.iloc[0]['track_name']} - {imprimir.iloc[0]['artists']} ({imprimir.iloc[0]['track_genre']})")

    musica_base = df_dados_normalizados.drop(columns='cluster').iloc[idx].values

    # Filtra por cluster
    subset = df_dados_normalizados[df_dados_normalizados['cluster'] == cluster_id].copy()
    
    # Filtra por gênero
    subset = subset[df_musicas.loc[subset.index, 'track_genre'] == genero_base]

    # Calcula a distância
    subset['distancia'] = subset.drop(columns='cluster').apply(
        lambda row: cosine(row.values, musica_base), axis=1
    )

    resultado_idx = subset.sort_values('distancia').index
    recomendacoes = df_musicas.iloc[resultado_idx][['track_name', 'artists', 'track_genre']]
    recomendacoes = recomendacoes[recomendacoes.index != idx]

    return recomendacoes.head(10)


Buscar música por artista

In [31]:
nome_artista = "Highly Suspect"
df_musicas[df_musicas['artists'] == nome_artista][['track_id','artists', 'track_name', 'album_name']]

,track_id,artists,track_name,album_name
2476,4z1N1HpvoBnptR3zP7Q8Ai,Highly Suspect,Natural Born Killer,Rock - Best of 2022
39420,78wMThFTNKThsWq19o0F1H,Highly Suspect,Natural Born Killer,Natural Born Killer/Pink Lullabye
39425,7yoBjKO6cCnK3zV8gr1k4e,Highly Suspect,Lydia,Mister Asylum
39549,1euszbbvkUzQzIhZUWFmXG,Highly Suspect,My Name Is Human,The Boy Who Died Wolf
39819,4nMUPQeTor23d8RxiiJO9W,Highly Suspect,Natural Born Killer,The Midnight Demon Club


## Recomendação

In [19]:
id_musica = "0EYOdF5FCkgOJJla8DI2Md"

Robust Scale

In [20]:
recomendar_musicas(id_musica,Drs,Mrs)

Música: B.Y.O.B. - System Of A Down


,track_name,artists,track_genre
4763,往け,LiSA,anime
55553,Blood Sweat & Tears,BTS,k-pop
34048,Rollin' (Air Raid Vehicle),Limp Bizkit,funk
55047,Blood Sweat & Tears,BTS,k-pop
39785,Crazy Bitch,Buckcherry,grunge
55222,Uchimandai,Vijay Antony;Krishna Iyer;Shoba Chandrasekar;C...,k-pop
59971,Welcome to the Family,Avenged Sevenfold,metal
31042,Kamli Wale,Nusrat Fateh Ali Khan;A1Melodymaster,folk
66665,Boomerang,The Summer Set,piano
69028,Angels (feat. Kat Nestel) - Radio Edit,Vicetone;Kat Nestel,progressive-house


In [21]:
recomendar_musicas_por_genero(id_musica,Drs,Mrs)

Música: B.Y.O.B. - System Of A Down (alternative)


,track_name,artists,track_genre
3201,Duality,Slipknot,alternative
3133,Chop Suey!,System Of A Down,alternative
3269,Afterlife,Avenged Sevenfold,alternative
3137,Faint,Linkin Park,alternative
3267,From the Inside,Linkin Park,alternative
3351,Faint,Linkin Park,alternative
3107,Bring Me To Life,Evanescence,alternative
3001,In the End,Linkin Park,alternative
3394,abc (The Wild Remix),GAYLE;The Wild,alternative
3294,Psychosocial,Slipknot,alternative


Min Max

In [22]:
recomendar_musicas(id_musica,Dminmax,Mminmax)

Música: B.Y.O.B. - System Of A Down


,track_name,artists,track_genre
2614,Eye Of The Storm,Watt White,alt-rock
2007,Unstoppable,The Score,alt-rock
60208,Two Weeks,All That Remains,metalcore
42459,You Really Got Me - 2015 Remaster,Van Halen,hard-rock
70585,Stupid for You,Waterparks,punk-rock
30686,Cemetery Drive,My Chemical Romance,emo
59971,Welcome to the Family,Avenged Sevenfold,metal
3051,CASTLE OF GLASS,Linkin Park,alternative
59940,Unholy Confessions,Avenged Sevenfold,metal
59915,You Give Love A Bad Name,Bon Jovi,metal


In [23]:
recomendar_musicas_por_genero(id_musica,Dminmax,Mminmax)

Música: B.Y.O.B. - System Of A Down (alternative)


,track_name,artists,track_genre
3051,CASTLE OF GLASS,Linkin Park,alternative
3207,Hail to the King,Avenged Sevenfold,alternative
3267,From the Inside,Linkin Park,alternative
3269,Afterlife,Avenged Sevenfold,alternative
3394,abc (The Wild Remix),GAYLE;The Wild,alternative
3131,New Gold (feat. Tame Impala and Bootie Brown),Gorillaz;Tame Impala;Bootie Brown,alternative
3119,Bubblegum Bitch,MARINA,alternative
3350,A Little Piece of Heaven,Avenged Sevenfold,alternative
3322,Lying from You,Linkin Park,alternative
3154,Breaking the Habit,Linkin Park,alternative


Max Abs

In [24]:
recomendar_musicas(id_musica,Dmaxabs,Mmaxabs)

Música: B.Y.O.B. - System Of A Down


,track_name,artists,track_genre
2007,Unstoppable,The Score,alt-rock
2614,Eye Of The Storm,Watt White,alt-rock
60208,Two Weeks,All That Remains,metalcore
70585,Stupid for You,Waterparks,punk-rock
59915,You Give Love A Bad Name,Bon Jovi,metal
3051,CASTLE OF GLASS,Linkin Park,alternative
59971,Welcome to the Family,Avenged Sevenfold,metal
42459,You Really Got Me - 2015 Remaster,Van Halen,hard-rock
30686,Cemetery Drive,My Chemical Romance,emo
4763,往け,LiSA,anime


In [25]:
recomendar_musicas_por_genero(id_musica,Dmaxabs,Mmaxabs)

Música: B.Y.O.B. - System Of A Down (alternative)


,track_name,artists,track_genre
3051,CASTLE OF GLASS,Linkin Park,alternative
3207,Hail to the King,Avenged Sevenfold,alternative
3267,From the Inside,Linkin Park,alternative
3394,abc (The Wild Remix),GAYLE;The Wild,alternative
3269,Afterlife,Avenged Sevenfold,alternative
3131,New Gold (feat. Tame Impala and Bootie Brown),Gorillaz;Tame Impala;Bootie Brown,alternative
3119,Bubblegum Bitch,MARINA,alternative
3350,A Little Piece of Heaven,Avenged Sevenfold,alternative
3322,Lying from You,Linkin Park,alternative
3154,Breaking the Habit,Linkin Park,alternative


Standart

In [26]:
recomendar_musicas(id_musica,Dstand,Mstand)

Música: B.Y.O.B. - System Of A Down


,track_name,artists,track_genre
4763,往け,LiSA,anime
55047,Blood Sweat & Tears,BTS,k-pop
59971,Welcome to the Family,Avenged Sevenfold,metal
2614,Eye Of The Storm,Watt White,alt-rock
60208,Two Weeks,All That Remains,metalcore
55553,Blood Sweat & Tears,BTS,k-pop
39785,Crazy Bitch,Buckcherry,grunge
39159,Soldiers,Otherwise,grunge
70585,Stupid for You,Waterparks,punk-rock
59915,You Give Love A Bad Name,Bon Jovi,metal


In [27]:
recomendar_musicas_por_genero(id_musica,Dstand,Mstand)

Música: B.Y.O.B. - System Of A Down (alternative)


,track_name,artists,track_genre
3051,CASTLE OF GLASS,Linkin Park,alternative
3207,Hail to the King,Avenged Sevenfold,alternative
3269,Afterlife,Avenged Sevenfold,alternative
3001,In the End,Linkin Park,alternative
3154,Breaking the Habit,Linkin Park,alternative
3107,Bring Me To Life,Evanescence,alternative
3321,Talking to Myself,Linkin Park,alternative
3179,New Divide,Linkin Park,alternative
3292,Aerials,System Of A Down,alternative
3268,Wake Up,Elaine,alternative


Sem normalizar

In [28]:
recomendar_musicas(id_musica,df_dados,df_musicas)

Música: B.Y.O.B. - System Of A Down


,track_name,artists,track_genre
31004,Little Talks,Of Monsters and Men,folk
59968,Freak On a Leash,Korn,metal
70507,Self Esteem,The Offspring,punk-rock
57896,Diles,Bad Bunny;Ozuna;Farruko;Arcangel;Ñengo Flow;DJ...,latino
45335,Without Me,Eminem,hip-hop
49210,Bags,Clairo,indie-pop
42577,Mo Money Mo Problems (feat. Puff Daddy & Mase)...,The Notorious B.I.G.;Mase;Diddy,hardcore
10326,Bleeding Love,Leona Lewis,british
48215,"Deva Deva (From ""Brahmastra"")",Pritam;Arijit Singh;Amitabh Bhattacharya;Jonit...,indian
2950,Time to Pretend,MGMT,alt-rock


In [29]:
recomendar_musicas_por_genero(id_musica,df_dados,df_musicas)

Música: B.Y.O.B. - System Of A Down (alternative)


,track_name,artists,track_genre
3399,Unsainted,Slipknot,alternative
3207,Hail to the King,Avenged Sevenfold,alternative
3140,Nightcall,Kavinsky,alternative
3179,New Divide,Linkin Park,alternative
3303,Behind Blue Eyes,Limp Bizkit,alternative
3300,It Won't Stop (feat. Chris Brown) - EP Version,Sevyn Streeter;Chris Brown,alternative
3294,Psychosocial,Slipknot,alternative
3398,Before I Forget,Slipknot,alternative
3301,Odd Look,Kavinsky;The Weeknd,alternative
3392,Zombie,Bad Wolves,alternative
